In [ ]:
Устанавливаем дополнительные библиотеки

In [2]:
!pip3 install tensorflow
!pip3 install opencv-python
!pip3 install keras-segmentation

     |████████████████████████████████| 479.6 MB 8.2 kB/s  eta 0:00:01     |███████████████████████████▍    | 410.2 MB 9.1 MB/s eta 0:00:08
     |████████████████████████████████| 17.3 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 22.9 MB 182 kB/s eta 0:00:01
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 5.4 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.8 MB 7.1 MB/s eta 0:00:01
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
     |████████████████████████████████| 133 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 7.5 MB/s eta 0:00:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 2.4 MB 8.2 MB/s eta 0:00:01███████████████  | 2.2 MB 8.2 MB/s eta 0:00:01
     |█████████

In [ ]:
Библиотеки, необходимые для этого блокнота

In [9]:
import cv2
import os
import cv2
import glob
import json
import numpy as np
import tensorflow as tf
from keras.layers import *
from keras_segmentation.models.model_utils import get_segmentation_model

In [ ]:
Загружаем датасет и обученные веса.

In [5]:
!wget --no-check-certificate  'https://docs.google.com/uc?export=download&id=1dsdW4KgEH_Rr9rU468hUbcKc1CnYeU9w' -O MLtest_dataset.tar.xz
!tar -xvf MLtest_dataset.tar.xz 

!wget --no-check-certificate \
 'https://docs.google.com/uc?export=download&id=1ttWSv-Sd5hdhVPcxpbWDY3ZbjS51_l1d' -O best.h5
 

--2024-02-21 16:27:58--  https://docs.google.com/uc?export=download&id=1dsdW4KgEH_Rr9rU468hUbcKc1CnYeU9w
Распознаётся docs.google.com (docs.google.com)… 64.233.165.194, 2a00:1450:4010:c07::c2
Подключение к docs.google.com (docs.google.com)|64.233.165.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 303 See Other
Адрес: https://drive.usercontent.google.com/download?id=1dsdW4KgEH_Rr9rU468hUbcKc1CnYeU9w&export=download [переход]
--2024-02-21 16:27:59--  https://drive.usercontent.google.com/download?id=1dsdW4KgEH_Rr9rU468hUbcKc1CnYeU9w&export=download
Распознаётся drive.usercontent.google.com (drive.usercontent.google.com)… 64.233.162.132, 2a00:1450:4010:c05::84
Подключение к drive.usercontent.google.com (drive.usercontent.google.com)|64.233.162.132|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 68710648 (66M) [application/octet-stream]
Сохранение в: «MLtest_dataset.tar.xz»

MLtest_dataset.tar. 100%[===================>]  65

In [ ]:
Предварительная обработка и разделение данных

In [10]:
if not os.path.exists("annotations_prepped_train"):
    os.makedirs("annotations_prepped_train")

if not os.path.exists("images_prepped_train"):
    os.makedirs("images_prepped_train")
    
if not os.path.exists("content"):
    os.makedirs("content")

color = {'Mixed': (1, 1, 1), 'Dense': (2, 2, 2), 'Diffuse': (3, 3, 3)}
annotation = {}

for v in glob.glob('MLtest_dataset/MLtest_json (*).json'):
    with open(v, 'r') as f:
        json_annotation = json.load(f)
        for key, cap in json_annotation.items():
            if key in annotation:
                annotation[key]['regions'] += cap['regions']
            else:
                annotation[key] = cap

for key, value in annotation.items():
    name_photo = f"MLtest_dataset/{value['filename']}"
    try:
        img = cv2.imread(name_photo)
        msk_img = np.zeros((768, 768, 3), dtype='uint8')
        for b in value['regions']:
            crd = b["shape_attributes"]
            if 'Type' in b["region_attributes"]:
                x_min = int((crd["x"] * msk_img.shape[1]) / img.shape[1])
                y_min = int((crd["y"] * msk_img.shape[0]) / img.shape[0])
                x_max = int(((crd["width"] + crd["x"]) * msk_img.shape[1]) / img.shape[1])
                y_max = int(((crd["height"] + crd["y"]) * msk_img.shape[0]) / img.shape[0])
                box = np.array([[[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]]], dtype=np.int32)
                cv2.fillPoly(msk_img, box, color[b["region_attributes"]["Type"]])
        img = cv2.resize(img, (768, 768))
        cv2.imwrite('images_prepped_train/' + value['filename'].replace('jpg', 'png'), img)
        cv2.imwrite('annotations_prepped_train/' + value['filename'].replace('jpg', 'png'), msk_img)
    except Exception as e:
        print(f'err {e} {name_photo}')

[ WARN:0@246.082] global loadsave.cpp:248 findDecoder imread_('MLtest_dataset/чашка 1.jpg'): can't open/read file: check file path/integrity


err OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
 MLtest_dataset/чашка 1.jpg


In [ ]:
Модель нейросети с некоторыми настраиваемыми слоями

In [14]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # 0-GPU -1-CPU
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
#session = tf.compat.v1.keras.backend.get_session()


def get_image_arr(path, width, height, imgNorm="sub_mean", odering='channels_first'):
    if type(path) is np.ndarray:
        img = path
    else:
        img = cv2.imread(path, 1)

    if imgNorm == "sub_and_divide":
        img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    elif imgNorm == "sub_mean":
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        img = img[:, :, ::-1]
    elif imgNorm == "divide":
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img = img / 255.0

    if odering == 'channels_first':
        img = np.rollaxis(img, 2, 0)
    return img


def predict_my(model=None, inp=None):
    assert len(inp.shape) == 3, "Image should be h,w,3 "
    output_width = model.output_width
    output_height = model.output_height
    input_width = model.input_width
    input_height = model.input_height
    n_classes = model.n_classes
    x = get_image_arr(inp, input_width, input_height, odering='channels_last')
    pr = model.predict(np.array([x]))[0]
    pr = pr.reshape((output_height, output_width, n_classes)).argmax(axis=2)   
    return np.array(pr, dtype=np.uint8)


def railway_unet(frame, model,n_classes, limit_box_width_height, show=True):
    img = predict_my(model=model, inp=frame)
    color = {1: (255, 100, 0), 2: (0, 100, 255), 3: (5, 255, 0)}
    account_classes = {1: 0, 2: 0, 3: 0}

    for c in range(1, n_classes, 1):
        only_cat_hsv = cv2.inRange(img, c, c)
        contours, hierarchy = cv2.findContours(only_cat_hsv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            box = cv2.boundingRect(cnt)
            if clinket_rail(box, limit_box_width_height):
                frame = cv2.rectangle(frame, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color[c], 5)
                account_classes[c] += 1
    if show:
        cv2.imshow('h1', frame)
        print(account_classes)
        cv2.waitKey(0)
    return f'Mixed: {account_classes[1]}, Dense: {account_classes[1]}, Diffuse: {account_classes[1]}.'


def clinket_rail(box, limit):
    return box[2] > limit[0] and box[3] > limit[1]


def net(n_classes, height, width, train, epochs):  # create a neural network for the segmentation of number
    img_input = Input(shape=(height, width, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    up0 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv41 = Conv2D(128, (3, 3), activation='relu', padding='same')(up0)
    conv41 = Dropout(0.2)(conv41)
    conv41 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv41)
    up1 = concatenate([UpSampling2D((2, 2))(conv41), conv2], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    out = Conv2D(n_classes, (1, 1), padding='same')(conv5)

    model = get_segmentation_model(img_input, out)  # this would build the segmentation model
    if train == True:
        model.train(
            train_images="images_prepped_train/",
            train_annotations="annotations_prepped_train/",
            checkpoints_path="content/", epochs=epochs)
        model.save('last.h5')

    return model


n_classes = 4
height = 768
width = 768

model = net(n_classes=n_classes, height=height, width=width, train=False, epochs=2)
print(model.summary())




Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 768, 768, 3)]        0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 768, 768, 32)         896       ['input_2[0][0]']             
                                                                                                  
 dropout_7 (Dropout)         (None, 768, 768, 32)         0         ['conv2d_15[0][0]']           
                                                                                                  
 conv2d_16 (Conv2D)          (None, 768, 768, 32)         9248      ['dropout_7[0][0]']           
                                                                                            

In [ ]:
Обучение
Мне потребовалось некоторое время, чтобы обучить модель (24 часа на четырех внешних графических процессорах). 
Так что просто напишем код для обучения и после этого загрузим обученную модель.

In [ ]:
model = net(n_classes=n_classes, height=height, width=width, train=True, epochs=200)

In [18]:
model.load_weights('best.h5')
frame = cv2.imread('images_prepped_train/чашка 11.png')
frame = cv2.resize(frame, (width, height))
print(frame.shape)



(768, 768, 3)


In [ ]:
limit_box_width_height = [10, 10]
account_classes = railway_unet(frame, model, n_classes, limit_box_width_height)